# Модуль2

pip install xgboost

In [60]:
import pandas as pd
import numpy as np
import joblib 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_curve, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [39]:
train_df=pd.read_csv("data/full_train_df.csv", index_col='session_id')


In [40]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 1 to 253561
Data columns (total 14 columns):
site1      253561 non-null int64
site2      253561 non-null int64
site3      253561 non-null int64
site4      253561 non-null int64
site5      253561 non-null int64
site6      253561 non-null int64
site7      253561 non-null int64
site8      253561 non-null int64
site9      253561 non-null int64
site10     253561 non-null int64
morning    253561 non-null int64
day        253561 non-null int64
evening    253561 non-null int64
target     253561 non-null int64
dtypes: int64(14)
memory usage: 29.0 MB


In [42]:
Y=train_df['target']
X=train_df.drop('target', axis=1)

Разделим выборку на обучающую и тестовую в соотношении 70/30. Выборка имеет 253561 наблюдения, это достаточно много и позволит оставить для тестирования 30%.

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,   test_size=0.3, random_state=42)

В качестве метрики выберем AUC-ROC. Так как в данной задаче имеем дело с несбалансированной выборкой. Критерий AUC-ROC устойчив к несбалансированным классам 

In [53]:
# Обучаем  логистическую регрессию

lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [54]:
y_pred_lr = lr.predict(X_test)

In [55]:
roc_auc_score(y_test, y_pred_lr)

0.5

In [68]:
# Обучаем случайный лес
RF = RandomForestClassifier(random_state=42)
RF.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [64]:
y_pred_rf = RF.predict(X_test)

In [65]:
roc_auc_score(y_test, y_pred_rf)

0.5154198018163912

In [69]:
# Обучаем XGB
GB=XGBClassifier(random_state=42)
GB.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:42:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=8, num_parallel_tree=1,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=True, validate_parameters=1, verbosity=None)

In [66]:
y_pred_GB = GB.predict(X_test)

In [67]:
roc_auc_score(y_test, y_pred_GB)

0.6011257715684213

XGBoost показал лучшее какчество, чем логистическая регрессия и случайный лес